In [ ]:
#%%bash
# wget https://www.python.org/ftp/python/3.6.5/Python-3.6.5.tgz
# tar xvfz Python-3.8.10.tgz
# cd Python-3.8.10
# ./configure
# make
# sudo make install 

In [ ]:
%%bash
apt-get install g++ openjdk-11-jdk python-dev python3-dev
pip3 install JPype1

Reading package lists...
Building dependency tree...
Reading state information...
python-dev is already the newest version (2.7.15~rc1-1).
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
openjdk-11-jdk is already the newest version (11.0.15+10-0ubuntu0.18.04.1).
python3-dev is already the newest version (3.6.7-1~18.04).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
import os

# Install java
! apt-get install -y openjdk-11-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

!pip install --ignore-installed -q pyspark==2.4.4

openjdk version "11.0.15" 2022-04-19
OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1)
OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)


In [ ]:
!pip install --ignore-installed -q spark-nlp==2.5.1

In [ ]:
!pip install koalanlp

In [ ]:
import koalanlp
from koalanlp import API
from koalanlp.proc import Parser
from koalanlp.Util import initialize, finalize

In [ ]:
!pip install konlpy
%cd /content/drive/MyDrive/Model2(keyword_extract)
initialize(java_options="-Xmx4g", KKMA = "LATEST")

/content/drive/MyDrive/Model2(keyword_extract)


Exception: ignored

In [277]:
import pandas as pd
import json
import koalanlp
from koalanlp import API
from koalanlp.proc import Parser
from koalanlp.Util import initialize, finalize, PhraseTag

In [ ]:
class KnuSL:
  def __init__(self):
    sent_dic = pd.read_csv("/content/drive/MyDrive/Model2(keyword_extract)/data/감정단어사전0603.csv")
    sent_dic = sent_dic[['단어' , '감정정도M']].dropna()
    self.data_list = []
    for ques, label in zip(sent_dic['단어'], sent_dic['감정정도M']):
      data_box = []
      data_box.append(ques)
      data_box.append(str(label))
      self.data_list.append(data_box)
  

  def get_sent(self, wordname):	
    with open('data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
      data = json.load(f)

    result = ['None','None']	
    for i in range(0, len(data)):
      word_roots = data[i]['word']

      if wordname == word_roots:
        result.pop()
        result.pop()
        result.append(word_roots)
        result.append(data[i]['polarity'])	
        break
    r_word = result[0]
    s_word = result[1]

    return r_word , s_word

In [325]:
#의존 관계 분석
def find_keyword(key_word,sentence):
    find_word = []
    parser = Parser(API.KKMA)
    sentences = parser(sentence)
    noun_tag = ['NNG' , 'NNP' , 'NP' , 'NR']

    for sent in sentences:
        dependencies = sent.getDependencies()
        if len(dependencies) > 0:
            for edge in dependencies:
              if key_word in str(edge.getGovernor()) : 
                edge_parsing = str(edge).split('→')[1].split('=')[1]
                edge_parsing = edge_parsing.lstrip().split("'")[0]
                edge_parsing = edge_parsing.split('+')

                if len(edge_parsing) == 1:
                  if edge_parsing[0].split('/')[1] in noun_tag: #부사격 조사의 종착지
                    find_word.append((edge_parsing[0]))
                    continue
                  else: continue
                n_value , j_value =edge_parsing[0].split('/')[0] , edge_parsing[1].split('/')[1]
    
   
                if j_value == 'JX' or j_value == 'JKS': #주격 조사 
                  #find_word.append(('주어' , n_value))
                  pass
                elif j_value == 'JKO':
                  find_word.append((n_value))
                elif j_value == 'JKB' or j_value == 'ECD' or j_value == 'EC' or j_value == 'JKM': #부사격 조사
                  pos_list = kkma.nouns(sentence)   
                  for noun in pos_list:
                    if noun == n_value:
                      break
                    else : find_word.append((noun))

    return find_word

In [326]:
from konlpy.tag import Kkma
kkma = Kkma()
def keyword(sentence):
  parsing_words = kkma.pos(sentence)

  ksl = KnuSL()
  sent_list = []
  for word_pos in parsing_words:
    if word_pos[1] == 'VV'or word_pos[1] == 'VA':    #elif word_pos[1] == 'NNG'  or word_pos[1] == 'MAG':
      word = word_pos[0] + '다'
    elif word_pos[1] == 'NNG'  or word_pos[1] == 'MAG':
      word = word_pos[0]
    else: continue

    r ,s = ksl.get_sent(word)
    if r != 'None':
      sent_list.append((r,s))

  if len(sent_list) == 0:
    print("출력 x")
    return None , None

  max_sent = max(sent_list , key = lambda x:x[1])
  keyword_list = find_keyword( kkma.pos(max_sent[0])[0][0] , sentence)
  return keyword_list 

In [327]:
example_list = []

In [328]:
sentence = "나는 너가 공부를 시작했기 때문에 기쁘다"

In [329]:
keyword(sentence)

['나', '너', '공부']

In [330]:
example_list.append(keyword(sentence))

In [ ]:
parsing_words = kkma.pos("어제 본 너는 너무 슬퍼보였어")
parsing_words

[('나', 'NP'),
 ('는', 'JX'),
 ('너', 'NP'),
 ('때문', 'NNB'),
 ('에', 'JKM'),
 ('짜증나', 'VV'),
 ('ㄴ다', 'EFN'),
 ('.', 'SF')]

In [ ]:
finalize()

True